In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math
import csv

<br></br>

# Get a list of first 50 uesrs

In [2]:
all_account_ids = []

In [3]:
num_user_pages = 5 

for page in range(num_user_pages):
    url = f'https://ratetea.com/people.php?&start={page*10}#page'
    user_info = requests.get(url)
    soup_info = BeautifulSoup(user_info.content, 'html.parser')
    account_ids=[a['href'].split('=')[1] for a in soup_info.select('a[href]') if a['href'].startswith('/profile/?account_id=')]
    all_account_ids += account_ids

In [5]:
len(all_account_ids)

50

<br></br>

# Get Tea Data for all Users

In [6]:
all_tea_data = []

In [ ]:
it = 0
# for account_id in all_account_ids:
for account_id in all_account_ids:
    it += 1
    try:
        # first page ratings
        url = 'https://ratetea.com/profile/ratings.php?account_id={}&brand_id=0&style_id=0&region_id=0&organic=&fairtrade=&loose=&caffeine=&o1=&start={}#page'
        r = requests.get(url.format(account_id, 0))
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # get total number of pages
        s = soup.find('p')
        num_reviews = int(str(s).split('Reviewed ')[1].split(' tea')[0])
        num_pages = math.ceil(num_reviews/20) # if num pages x, then there are 0-19, 20-40, ..., x*20 -
       
        # get data from all pages
        for page_id in range(num_pages):
            start_page = page_id * 20
            
            # get page data
            url = 'https://ratetea.com/profile/ratings.php?account_id={}&brand_id=0&style_id=0&region_id=0&organic=&fairtrade=&loose=&caffeine=&o1=&start={}#page'
            r = requests.get(url.format(account_id, start_page))
            soup = BeautifulSoup(r.content, 'html.parser')
            
            # get Account Id, Tea Link, Tea Name
            tea_data = [[account_id, 'https://ratetea.com/' + a['href'], a.text] for a in soup.select('a[href]') if a['href'].startswith('/review')]
            
            # add to list
            all_tea_data += tea_data
        
        print(f'User {account_id} added successfully; {it} / {num_user_pages*10}')
    except:
        print(f'Encounter issue with user {account_id}')

User 1 added successfully; 1 / 730
User 2282 added successfully; 2 / 210
User 833 added successfully; 3 / 170
User 1398 added successfully; 4 / 170


<br></br>

# Save Tea and User data

In [ ]:
with open('user_tea_pairs.csv', 'w', encoding='utf-8') as f:
    write = csv.writer(f)
    write.writerows(all_tea_data)

In [ ]:
df = pd.read_csv('user_tea_pairs.csv', delimiter=',', header=None)
df